In [61]:
import numpy as np
import pandas as pd
import openpyxl 
import os

print(os.listdir("../CCP/data"))

['Q4-2017', 'Q4-2018', 'ccp-austria-q1-q4-2016.xlsx', 'Q3-2017', 'euro-ccp-q4-2015-q2-2018.xlsx', 'Q2-2018', 'ccp-austria-q1-q2-2018.xlsx', 'Q2-2016', 'Q3-2018', 'kursy-walutowe-CCP.xlsx', 'Q3-2016', 'ccp-austria-q1-q4-2017.xlsx', 'Q4-2015', 'Q1-2017', 'Q2-2017', 'Q1-2018', 'Q1-2016', 'Q4-2016']


In [62]:
#basic
b_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='AggregatedDataFile') 
#files in use
a_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='Nasdaq_DataFile_4_4a_2017_Q3')
c_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='Nasdaq_DataFile_6_1_2017_Q3')
d_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='Nasdaq_DataFile_6.2_2017_Q3')
e_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='Nasdaq_DataFile_18_2_2017_Q3')
g_file = pd.read_excel('../CCP/data/Q3-2017/Nasdaq-q3-2017.xlsx', sheet_name='Nasdaq_DataFile_23_2017_Q3')
ex_rate = pd.read_excel('../CCP/data/kursy-walutowe-CCP.xlsx', sheet_name='Arkusz1')

quarter = 'Q3-2017'

In [63]:
ex_rate.head()

,Exchange rates,Q4-2015,Q1-2016,Q2-2016,Q3-2016,Q4-2016,Q1-2017,Q2-2017,Q3-2017,Q4-2017,Q1-2018,Q2-2018
0,EUR/SEK,9.1895,9.2253,9.4242,9.6210,9.5525,9.5322,9.6398,9.6490,9.8438,10.2843,10.4530
1,EUR/NOK,9.6030,9.4145,9.3008,8.9865,9.0863,9.1683,9.5713,9.4125,9.8403,9.6770,9.5115
2,EUR/DKK,7.4626,7.4512,7.4393,7.4513,7.4344,7.4379,7.4366,7.4423,7.4449,7.4530,7.4525
3,EUR/USD,1.0887,1.1385,1.1102,1.1161,1.0541,1.0691,1.1412,1.1806,1.1993,1.2321,1.1658
4,EUR/CHF,1.0835,1.0931,1.0867,1.0876,1.0739,1.0696,1.0930,1.1457,1.1702,1.1779,1.1569


In [64]:
#extra col 'ex_rate' in g_file for counting '23.1.2'
for row in range(len(g_file)):
    if g_file.loc[row, 'Currency'] == 'SEK':
        g_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/SEK'])][quarter].values
    elif g_file.loc[row, 'Currency'] == 'NOK':
        g_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/NOK'])][quarter].values
    elif g_file.loc[row, 'Currency'] == 'USD':
        g_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/USD'])][quarter].values
    elif g_file.loc[row, 'Currency'] == 'DKK':
        g_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/DKK'])][quarter].values
    elif g_file.loc[row, 'Currency'] == 'EUR':
        g_file.loc[row, 'ex_rate'] = 1
    else:
        pass


In [65]:
#recalculate g_file to EUR (so recalculations for '23.1.2' and '23.2.1' in f_file are skipped)
g_file['23.1.2'] = g_file['23.1.2'] // g_file['ex_rate']
g_file['23.2.1'] = g_file['23.2.1'] // g_file['ex_rate']

In [68]:
#final file
f_file = pd.DataFrame({'CCP':'Nasdaq OMX'}, index=[0,1,2])
f_file['quarter'] = quarter
f_file['clearning service'] = b_file['ReportLevelIdentifier']
f_file['currency'] = b_file['Currency']
f_file['4.1.1'] = b_file['4.1.1']
f_file['4.1.2'] = b_file['4.1.2']
f_file['4.1.3'] = b_file['4.1.3']
f_file['4.1.4'] = b_file['4.1.4']
f_file['4.1.5'] = b_file['4.1.5']
f_file['4.1.6'] = b_file['4.1.6']
f_file['4.1.7'] = b_file['4.1.7']
f_file['4.1.8'] = b_file['4.1.8']
f_file['4.1.9'] = b_file['4.1.9']
f_file['4.1.10'] = b_file['4.1.10']

In [47]:
#extra col in f_file for exchange rate
for row in range(len(f_file)):
    if f_file.loc[row, 'currency'] == 'SEK':
        f_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/SEK'])][quarter].values
    elif f_file.loc[row, 'currency'] == 'NOK':
        f_file.loc[row, 'ex_rate'] = ex_rate[ex_rate['Exchange rates'].isin(['EUR/NOK'])][quarter].values
    elif f_file.loc[row, 'currency'] == 'EUR':
        f_file.loc[row, 'ex_rate'] = 1
    else:
        pass

In [48]:
# 4.4.7.PDA & 4.4.7.MA
f_file['4.4.7.PDA'] = a_file[a_file.Description == 'PeakDayAmountInPast12Months'].reset_index()['4.4.7']
f_file['4.4.7.MA'] = a_file[a_file.Description == 'MeanAverageOverPrevious12Months'].reset_index()['4.4.7']

In [49]:
#6.6.1.CLIENT = 'Client_Net' + 'Client_Gross'
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '6.6.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Financial Markets'])]['6.1.1'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '6.6.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Commodities'])]['6.1.1'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '6.6.1.CLIENT'] = sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Seafood'])]['6.1.1'])
    else:
        pass
    

#6.6.1.TOTAL = 'House_Net' + 'Client_Net' + 'Client_Gross'    
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '6.6.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Financial Markets'])]['6.1.1'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '6.6.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Commodities'])]['6.1.1'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '6.6.1.TOTAL'] = sum(c_file[c_file.Description.isin(['House_Net', 'Client_Net', 'Client_Gross']) & c_file.ReportLevelIdentifier.isin(['Seafood'])]['6.1.1'])
    else:
        pass

In [50]:
#6.2.15.PRE & POST
f_file['6.2.15.PRE'] = d_file[d_file.Description == 'TotalIM_PreHaircut'].reset_index()['6.2.15']
f_file['6.2.15.POST'] = d_file[d_file.Description == 'TotalIM_PostHaircut'].reset_index()['6.2.15']

In [54]:
#number of CCP
for row in range(len(f_file)):
    try:
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.1.1'].values
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.1.1'].values
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '18.1.1.1'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.1.1'].values
    except:
        pass
    
for row in range(len(f_file)):
    try:
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.1.2'].values
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.1.2'].values
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '18.1.1.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.1.2'].values
    except:
        pass
    


In [69]:
#number od CCP participants (average)
for row in range(len(f_file)):
    try:
        if f_file.loc[row, 'clearning service'] == 'Financial Markets':
            f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Financial Markets')]['18.1.2.2'].values
        elif f_file.loc[row, 'clearning service'] == 'Commodities':
            f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Commodities')]['18.1.2.2'].values
        elif f_file.loc[row, 'clearning service'] == 'Seafood':
            f_file.loc[row, '18.1.2.2'] = b_file[(b_file.ReportLevel == 'Membership') & 
                                                   (b_file.ReportLevelIdentifier == 'Seafood')]['18.1.2.2'].values
    except:
        pass


In [38]:
#number of clients
f_file['19.1.1'] = b_file['19.1.1']
f_file['19.1.2'] = b_file['19.1.2']

In [15]:
#23.1.2.OTC and 23.1.2.ETD
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.1.2'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Commodities')]['23.1.2'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '23.1.2.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Seafood')]['23.1.2'])
    else:
        pass
    
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.1.2'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Commodities')]['23.1.2'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '23.1.2.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Seafood')]['23.1.2'])
    else:
        pass

In [16]:
#23.2.1.OTC and 23.2.1.ETD
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.2.1'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Commodities')]['23.2.1'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '23.2.1.OTC'] = sum(g_file[(g_file.Description.map(lambda x: 'OTC' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Seafood')]['23.2.1'])
    else:
        pass
    
for row in range(len(f_file)):
    if f_file.loc[row, 'clearning service'] == 'Financial Markets':
        f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Financial Markets')]['23.2.1'])
    elif f_file.loc[row, 'clearning service'] == 'Commodities':
        f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Commodities')]['23.2.1'])
    elif f_file.loc[row, 'clearning service'] == 'Seafood':
        f_file.loc[row, '23.2.1.ETD'] = sum(g_file[(g_file.Description.map(lambda x: 'ETD' in x) == True) & 
                                               (g_file.ReportLevelIdentifier == 'Seafood')]['23.2.1'])
    else:
        pass

In [23]:
#convert values to EUR for variable from '4.1.1' to '4.1.10' and 
#'4.4.7.PDA', '4.4.7.MA', '6.6.1.TOTAL', '6.6.1.CLIENT'
f_file['4.1.1'] = f_file['4.1.1'] // f_file['ex_rate']
f_file['4.1.2'] = f_file['4.1.2'] // f_file['ex_rate']
f_file['4.1.3'] = f_file['4.1.3'] // f_file['ex_rate']
f_file['4.1.4'] = f_file['4.1.4'] // f_file['ex_rate']
f_file['4.1.5'] = f_file['4.1.5'] // f_file['ex_rate']
f_file['4.1.6'] = f_file['4.1.6'] // f_file['ex_rate']
f_file['4.1.7'] = f_file['4.1.7'] // f_file['ex_rate']
f_file['4.1.8'] = f_file['4.1.8'] // f_file['ex_rate']
f_file['4.1.9'] = f_file['4.1.9'] // f_file['ex_rate']
f_file['4.1.10'] = f_file['4.1.10'] // f_file['ex_rate']
f_file['4.4.7.PDA'] = f_file['4.4.7.PDA'] // f_file['ex_rate']
f_file['4.4.7.MA'] = f_file['4.4.7.MA'] // f_file['ex_rate']
f_file['6.6.1.TOTAL'] = f_file['6.6.1.TOTAL'] // f_file['ex_rate']
f_file['6.6.1.CLIENT'] = f_file['6.6.1.CLIENT'] // f_file['ex_rate']
f_file['6.2.15.PRE'] = f_file['6.2.15.PRE'] // f_file['ex_rate']
f_file['6.2.15.POST'] = f_file['6.2.15.POST'] // f_file['ex_rate']
f_file['currency'] = 'EUR'
print(f_file)

          CCP  clearning service currency      4.1.1  4.1.2      4.1.3  \
0  Nasdaq OMX  Financial Markets      EUR  1020372.0   -0.0  1399436.0   
1  Nasdaq OMX        Commodities      EUR  4338089.0   -0.0  6571573.0   
2  Nasdaq OMX            Seafood      EUR    51199.0   -0.0    15765.0   

        4.1.4        4.1.5       4.1.6       4.1.7      ...       \
0  25292900.0   28035164.0    189574.0    421042.0      ...        
1  97750000.0  117975262.0  50268661.0  23790196.0      ...        
2    545176.0     857479.0    367629.0    180410.0      ...        

    6.2.15.POST  18.1.1.1  18.1.1.2  18.2.1  19.1.1  19.1.2    23.1.2.OTC  \
0  4.214330e+08       NaN       NaN     NaN  4157.0    40.0  9.171018e+09   
1  1.949453e+09       NaN       NaN     NaN   234.0   179.0  0.000000e+00   
2  4.059909e+06       NaN       NaN     NaN    22.0    22.0  0.000000e+00   

     23.1.2.ETD    23.2.1.OTC    23.2.1.ETD  
0  2.854049e+09  2.988746e+11  8.627851e+09  
1  1.517950e+08  0.000000e+00

In [24]:
#drop 'ex_rate'
f_file.drop(columns=['ex_rate'])

,CCP,clearning service,currency,4.1.1,4.1.2,4.1.3,4.1.4,4.1.5,4.1.6,4.1.7,...,6.2.15.POST,18.1.1.1,18.1.1.2,18.2.1,19.1.1,19.1.2,23.1.2.OTC,23.1.2.ETD,23.2.1.OTC,23.2.1.ETD
0,Nasdaq OMX,Financial Markets,EUR,1020372.0,-0.0,1399436.0,25292900.0,28035164.0,189574.0,421042.0,...,4.214330e+08,NaN,NaN,NaN,4157.0,40.0,9.171018e+09,2.854049e+09,2.988746e+11,8.627851e+09
1,Nasdaq OMX,Commodities,EUR,4338089.0,-0.0,6571573.0,97750000.0,117975262.0,50268661.0,23790196.0,...,1.949453e+09,NaN,NaN,NaN,234.0,179.0,0.000000e+00,1.517950e+08,0.000000e+00,8.993856e+09
2,Nasdaq OMX,Seafood,EUR,51199.0,-0.0,15765.0,545176.0,857479.0,367629.0,180410.0,...,4.059909e+06,NaN,NaN,NaN,22.0,22.0,0.000000e+00,3.473974e+06,0.000000e+00,2.560207e+08


In [25]:
writer = pd.ExcelWriter('Nasdaq_Q3_2017.xlsx')
f_file.to_excel(writer, 'Sheet1')
writer.save()